In [ ]:
import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cobra.flux_analysis import single_gene_deletion
from molmass import Formula

%matplotlib inline

In [ ]:
model=cobra.io.read_sbml_model("D:/ScientificReports/iPN730.xml")
model.solver='glpk'
model.reactions.EX_cpd00092_e0.bounds=(0,1000)
model.reactions.EX_cpd19013_e0.bounds=(-1000,1000)
model.reactions.EX_cpd00027_e0.bounds=(-2.28,1000)
model.reactions.EX_cpd00007_e0.bounds=(-10,1000)
model.summary()

In [ ]:
bio=model.reactions.get_by_id('bio1')
met_pc=bio.metabolites
met_names=[]
met_coeff=[]
met_formula=[]
met_chemnames=[]
for met in met_pc:
    met_names.append(met.id)
    met_coeff.append(met_pc[met])
    met_chemnames.append(met.name)
    met_formula.append(met.formula)
met_formula[1]='C4140H7644O1300P100'
met_formula[9]='C3736H7172N100O1000P100'
met_formula[10]='C3636H7172N100O800P100'

In [ ]:
def cnratio(modelg):
    nh4=np.linspace(0.1,2.27,20)
    cn=((6*2.26)/(1*nh4)*(12/14))
    fit=[]
    for n in nh4:
        modelg.reactions.EX_cpd19013_e0.bounds=(-n,1000)
        fit.append(modelg.slim_optimize())
    cnval=cn[fit.index(max(fit))]
    return [cnval,max(fit)]

In [ ]:
def gr(p,metc,metn):
    if(metc<0):
        
        #Increase
        modelf=model.copy()
        biof=modelf.reactions.get_by_id('bio1')
        biof.add_metabolites({modelf.metabolites.get_by_id(metn):p*(-metc)})
        gr_plus10=modelf.slim_optimize()
        sgd_minimal=single_gene_deletion(model,method='FBA')
        sgd_minimal['SimulationTruth']=sgd_minimal['growth'].apply(lambda x:'Growth' if (round(x,5)!=0) else 'No Growth')
        ess_genp10=sgd_minimal[sgd_minimal['SimulationTruth']=='No Growth'].count()['growth']
        
#         optgr_p10=cnratio(modelf)
        
        #Decrease
        modelf=model.copy()
        biof=modelf.reactions.get_by_id('bio1')
        biof.add_metabolites({modelf.metabolites.get_by_id(metn):p*(metc)})
        gr_minus10=modelf.slim_optimize()
        sgd_minimal=single_gene_deletion(model,method='FBA')
        sgd_minimal['SimulationTruth']=sgd_minimal['growth'].apply(lambda x:'Growth' if (round(x,5)!=0) else 'No Growth')
        ess_genm10=sgd_minimal[sgd_minimal['SimulationTruth']=='No Growth'].count()['growth']
        #         optgr_m10=cnratio(modelf)
        
        return gr_plus10,gr_minus10,ess_genp10,ess_genm10

In [ ]:
def atomcount(formula,e):
    f=Formula(formula)
    get=list(f.composition())
    d=pd.DataFrame(get,columns=['Element','Count','Bla1','Bla2'])
    d.index=d['Element']
    return d['Count'][e]

In [ ]:
p=50
p=p/100
p10=[]
m10=[]
ess_p10=[]
ess_m10=[]
for metn,metc in zip(met_names,met_coeff):
    print('Current Biomass Precursor: ',metn)
    grp10,grm10,essp10,essm10=gr(p,metc,metn)
    p10.append(grp10)
    m10.append(grm10)
    ess_p10.append(essp10)
    ess_m10.append(essm10)
    print('Increase-Growth Rate: ',grp10)
    print('Decrease-Growth Rate: ',grm10)
    print('Increase-Essential Genes: ',essp10)
    print('Decrease-Essential Genes: ',essm10)

In [ ]:
df_sorted=pd.DataFrame({"Name":pd.Series(met_chemnames[:44]),"Increase":p10,"Decrease":m10,"Formula":met_formula[:44],'Coeff':met_coeff[:44]})
df_sorted['Diff']=df_sorted['Increase']-df_sorted['Decrease']
df_sorted['Formula']=df_sorted['Formula'].apply(lambda x: x[0:x.index('R')] if ('R' in x) else x)
df_sorted=df_sorted.sort_values(by='Diff',ascending=False)
df_sorted=df_sorted.reset_index(drop=True)

In [ ]:
#CountC/NRatio
df_sorted['C']=df_sorted['Formula'].apply(lambda x:(atomcount(x,'C')) if ('C' in x) else 0)
df_sorted['N']=df_sorted['Formula'].apply(lambda x:(atomcount(x,'N')) if ('N' in x) else 0)
df_sorted['Cmols']=df_sorted['C']*abs(df_sorted['Coeff'])
df_sorted['Nmols']=df_sorted['N']*abs(df_sorted['Coeff'])

In [ ]:
import matplotlib.ticker as ticker
fig,ax=plt.subplots(nrows=1,ncols=1,figsize=(12,20))
ax.bar(df_sorted['Name'],df_sorted['Diff'],bottom=df_sorted['Decrease'],color='r')
ax.set_ylabel('Biomass Precursors',fontsize=14)
ax.set_xlabel('Growth rate (1/hr)',fontsize=14)
ax.set_xlim([0.205,0.24])
plt.yticks(np.arange(0,44,1),rotation=40)
ax.set_yticklabels(met_chemnames,fontsize=16)
ax.plot(df_sorted['Increase'],'bo')
ax.plot(df_sorted['Decrease'],'yo')
plt.gca().invert_xaxis()

In [ ]:
fig.savefig("D:/ScientificReports/BiomassSensitivityNames.jpg",dpi=1200)

In [ ]:
df_sorted['CmolsInc']=df_sorted['Cmols']*1.5
df_sorted['CmolsDec']=df_sorted['Cmols']*0.5
df_sorted['NmolsInc']=df_sorted['Nmols']*1.5
df_sorted['NmolsDec']=df_sorted['Nmols']*0.5

df_sorted['C/NInc']=pd.Series()
df_sorted['C/NDec']=pd.Series()

for i in range(len(df_sorted)):
    df_sorted['C/NInc'][i]=(df_sorted['Cmols'].sum()+(df_sorted['CmolsInc'][i]-df_sorted['Cmols'][i]))/(df_sorted['Nmols'].sum()+(df_sorted['NmolsInc'][i]-df_sorted['Nmols'][i]))
    df_sorted['C/NDec'][i]=(df_sorted['Cmols'].sum()+(df_sorted['Cmols'][i]-df_sorted['CmolsDec'][i]))/(df_sorted['Nmols'].sum()+(df_sorted['Nmols'][i]-df_sorted['NmolsDec'][i]))
    

In [ ]:
df_sorted

In [ ]:
cobra.io.write_sbml_model(model,"D:/ScientificReports/iPN730.xml")